# Clean-up Resource

# 1. lab_01_training

### SageMaker Experiments

In [7]:
import time

from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent

In [8]:
experiment_name='xgboost-poc-1'  #<== 생성한 experiment_name을 넣어야 합니다.

In [9]:
def cleanup_sme_sdk(experiment):
    for trial_summary in experiment.list_trials():
        trial = Trial.load(trial_name=trial_summary.trial_name)
        for trial_component_summary in trial.list_trial_components():
            tc = TrialComponent.load(
                trial_component_name=trial_component_summary.trial_component_name)
            trial.remove_trial_component(tc)
            try:
                # comment out to keep trial components
                tc.delete()
            except:
                # tc is associated with another trial
                continue
            # to prevent throttling
            time.sleep(.5)
        trial.delete()
        experiment_name = experiment.experiment_name
    experiment.delete()
    print(f"\nExperiment {experiment_name} deleted")

In [10]:
experiment_to_cleanup = Experiment.load(
    # Use experiment name not display name
    experiment_name=experiment_name)

cleanup_sme_sdk(experiment_to_cleanup)


Experiment xgboost-poc-1 deleted


## S3 버킷 삭제

In [11]:
import sagemaker

In [12]:
sagemaker_session = sagemaker.session.Session()

In [13]:
bucket = sagemaker_session.default_bucket()
bucket

'sagemaker-us-east-1-051065130547'

In [14]:
!aws s3 rm s3://$bucket/xgboost

delete: s3://sagemaker-us-east-1-051065130547/xgboost


# 2. lab_03_pipeline 

## 2.1. 파이프라인 삭제


### 파이프라인 이름 검색
- "sagemaker-webinar" 이름으로 시작되는 파이프라인 정보 검색

In [ ]:
import boto3

client = boto3.client('sagemaker')

response = client.list_pipelines(
    PipelineNamePrefix='sagemaker-webinar',
    SortBy='Name',
    SortOrder='Descending',
    MaxResults=1
)

response

### 'sagemaker-webinar-pipeline-base' 파이프라인 삭제

In [ ]:
deleted_pipeline = 'sagemaker-webinar-pipeline-base'
response = client.delete_pipeline(
    PipelineName=deleted_pipeline,
)
response

### 'sagemaker-webinar-pipeline-base' 파이프라인 관련된 버킷 내용 삭제

In [ ]:
! aws s3 rm s3://{bucket}/{deleted_pipeline} --recursive

### 버킷의 첫번째 폴더 리스트
- 아래와 같은 모둔 폴더를 확인하신 후에 필요한 폴더는 삭제 하세요.

In [6]:
! aws s3 ls {bucket}

                           PRE Fraud-Advance-Eval-955a33142036ab10f5b2dc96b96cd633/
                           PRE Fraud-Advance-Evaluation-955a33142036ab10f5b2dc96b96cd633/
                           PRE Fraud-Advance-Preprocess-9eb3e47661f2f1f3db35bb21017985f2/
                           PRE Fraud-Basic-Preprocess-476f3bbbf986da0944ce9ca52d7c54b6/
                           PRE Fraud-Basic-Process-476f3bbbf986da0944ce9ca52d7c54b6/
                           PRE Fraud-Basic-Process-f65211cab06c0afa0f94d059ed66c9e6/
                           PRE FraudScratchProcess-f65211cab06c0afa0f94d059ed66c9e6/
                           PRE fraud2train/
                           PRE sagemaker-pipeline-step-by-step/
                           PRE sagemaker-webinar-pipeline-advanced/
                           PRE sagemaker-xgboost-2022-03-21-12-18-21-232/
                           PRE sagemaker-xgboost-2022-03-21-12-18-21-493/
                           PRE sagemaker-xgboost-2022-03-21-12-26-39-

## 2.2 앤드포인트 삭제 확인
- 아래와 같이 세이지 메이커 콘솔에 가서, 혹시 앤드포인트가 있나를 확인 하세요. 존재하면 아래 그림과 같이 삭제 하시기 바랍니다.

![delete_endpoint.png](img/delete_endpoint.png)

# 4. SageMaker Classic Notebook 삭제

## 4.1. CloudFormation 콘솔로 이동
- 아래와 같이 "서비스 창" 에 "Cloudf" 입력
- 그리고 CloudFormation 서비스 메뉴가 나오면 클릭

![cf_console.png](img/cf_console.png)

## 4.2. 스텍 선택 후에 삭제
- 아래와 같이 처음 설치했던 스텍 (예: sagemaker-notebook) 선택
- 그리고 삭제 버튼 클릭

![remove_stack.png](img/remove_stack.png)

## 4.3. 세이지 메이커 콘솔로 이동
- 왼쪽 메뉴의 "노트북 >> 노트북 인스턴스" 클릭
- 그리고 (2) 번의 "Stopping" 확인

![go_sm_console.png](img/go_sm_console.png)

## 4.4. 노트북 삭제 확인
- 아래와 같이 해당 노트북이 지워지면, 해당 노트북 인스턴스가 보이지 않습니다. 

![sure_notebook.png](img/sure_notebook.png)

#### 오늘 하루 동안 고생 많이 하셨습니다. 많은 배우셨기를 기원 합니다. 고맙습니다.